### Environment requirement

* Standard Library: 
    * [io](https://docs.python.org/2/library/io.html)
    * [time](https://docs.python.org/2/library/time.html)
    * [json](https://docs.python.org/2/library/json.html)

* Third Party
    * [requests](http://docs.python-requests.org/en/master/)
    * [spotipy](https://pypi.org/project/spotipy/): pip3 install spotipy

In [2]:
import spotipy
import spotipy.oauth2 as oauth2
import io, time, json
import requests
from pathlib import Path

In [3]:
def read_client_info(filepath):
    """
    Read the Spotify Client ID from file.
    
    Args:
        filepath (string): File containing API Key
    Returns:
        client_id (string): The client_id
    """
    return Path(filepath).read_text().strip()

def generate_token():
    """ Generate the token from client_id and client_secret """
    credentials = oauth2.SpotifyClientCredentials(
        client_id=read_client_info("client_id.txt"),
        client_secret=read_client_info("client_secret.txt"))
    token = credentials.get_access_token()
    return token


In [3]:
# def write_tracks(text_file, tracks):
#     with open(text_file, 'a') as file_out:
#         while True:
#             for item in tracks['items']:
#                 if 'track' in item:
#                     track = item['track']
#                 else:
#                     track = item
#                 try:
#                     track_url = track['external_urls']['spotify']
#                     file_out.write(track_url + '\n')
#                 except KeyError:
#                     print(u'Skipping track {0} by {1} (local only?)'.format(
#                             track['name'], track['artists'][0]['name']))
#             # 1 page = 50 results
#             # check if there are more pages
#             if tracks['next']:
#                 tracks = spotify.next(tracks)
#             else:
#                 break

# def write_playlist(username, playlist_id):
#     results = spotify.user_playlist(username, playlist_id,
#                                     fields='tracks,next,name')
#     text_file = u'{0}.txt'.format(results['name'], ok='-_()[]{}')
#     print(u'Writing {0} tracks to {1}'.format(
#             results['tracks']['total'], text_file))
#     tracks = results['tracks']
#     write_tracks(text_file, tracks)

In [9]:
# def get_list_of_category(token):
#     """
#     Make an authenticated request to the Spotify API to get a list of browse categories.
#     """
#     headers = {'Authorization': 'Bearer %s' % token}
#     params = {"limit" : 50, "offset" : 0}
#     res = requests.get("https://api.spotify.com/v1/browse/categories", headers=headers,params=params)
# #     print(res.text)
#     d = json.loads(res.text)
#     d = d['categories']
# #     print(d['total'])
# #     print(len(d['items']))
#     return d

# def get_all_categories_id(items):
#     res = []
#     for item in items:
#         res.append(item['href'])
#     return res
    
# def get_all_playlists_from_category(token, end_point):
#     """
#     Make an authenticated request to the Spotify API to get a list of playlists from the category.

#     """
#     import pdb
    
#     headers = {'Authorization': 'Bearer %s' % token}
# #     params = {"limit" : 50, "offset" : 0}
#     end_point = end_point+"/playlists/?offset=0&limit=50"
#     ret = []
#     while end_point:
# #         pdb.set_trace()
#         res = requests.get(end_point, headers=headers)
#         print(res.status_code)
#         if res.status_code == 200:
#             d = json.loads(res.text)
#             d = d['playlists']
#             items = d['items']
#             end_point = d['next']
#             for item in items:
#                 tmp_d = {}
#                 tmp_d['name'] = item['name']
#                 tmp_d['href'] = item['href']
#                 tmp_d['id'] = item['id']
#                 ret.append(tmp_d)
    
#     print(len(ret))
#     return ret

def get_response(token, uri):
    headers = {'Authorization': 'Bearer %s' % token}
    rsp = requests.get(uri, headers=headers)
    return rsp.status_code, rsp.text

def get_tracks_from_playlist(token, playlist_uri):
    """
    Make an authenticated request to the Spotify API to get all the tracks in a public playlist.
    """
    res = []
    status_code, txt = get_response(token, playlist_uri)
    if status_code == 200:
        d = json.loads(txt)
        tracks = d['tracks']
        while True:
            for item in tracks['items']:
                track_name = item['track']['name']
                res.append(track_name)
            if tracks['next']:
                tracks = spotify.next(tracks)
            else:
                break
    return res

def get_users_public_playlist(token, user_id):
    """
    Make an authenticated request to the Spotify API to get users public playlist.
    """
    res = []
    nxt = "https://api.spotify.com/v1/users/" + user_id + "/playlists/?limit=50&offset=0"
    while nxt:
        status, txt = get_response(token, nxt)
        if status == 200:
            d = json.loads(txt)
            nxt = d['next']
            items = d['items']
            for item in items:
                tmp_d = {}
                tmp_d['id'] = item['id']
                tmp_d['href'] = item['href']
                tmp_d['name'] = item['name']
                track_names = get_tracks_from_playlist(token,item['href'])
                res.append(tmp_d)
        else:
            return None
    return res

In [5]:
token = generate_token()
spotify = spotipy.Spotify(auth=token)

# example playlist
playlists = get_users_public_playlist(token,"#obviously")
len(playlists)

TypeError: object of type 'NoneType' has no len()

In [27]:
with open('playlists.json', 'w') as fp:
    json.dump(playlists, fp)

In [ ]:
token = generate_token()
spotify = spotipy.Spotify(auth=token)
tracks = get_tracks_from_playlist(token,"https://api.spotify.com/v1/playlists/0HPyXnRDPpoDkY15u9d8Tl")
len(tracks)

--Return--
> <ipython-input-9-bc370e3d4d94>(65)get_tracks_from_playlist()->None
-> pdb.set_trace()
(Pdb) d
*** Newest frame
(Pdb) print(d)
{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/0HPyXnRDPpoDkY15u9d8Tl'}, 'followers': {'href': None, 'total': 0}, 'href': 'https://api.spotify.com/v1/playlists/0HPyXnRDPpoDkY15u9d8Tl', 'id': '0HPyXnRDPpoDkY15u9d8Tl', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273e198b6b9c5bfe013458e8ec9', 'width': 640}], 'name': 'liked', 'owner': {'display_name': 'YIYANG GE', 'external_urls': {'spotify': 'https://open.spotify.com/user/kkgc9fu3tu1cjmm721i2khqhz'}, 'href': 'https://api.spotify.com/v1/users/kkgc9fu3tu1cjmm721i2khqhz', 'id': 'kkgc9fu3tu1cjmm721i2khqhz', 'type': 'user', 'uri': 'spotify:user:kkgc9fu3tu1cjmm721i2khqhz'}, 'primary_color': None, 'public': True, 'snapshot_id': 'Miw0NWFiMzM0ZWFmMzE1NTdhNTdmNDZjMTA1NDEzYTJjMjBhYTI5ODQ3', 'tracks': {'href': 'https://api.spo

(Pdb) tracks['items'][0]['name']
*** KeyError: 'name'
(Pdb) tracks['items'][0]['href']
*** KeyError: 'href'
(Pdb) tracks['items'][0]
{'added_at': '2019-11-25T20:48:56Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/kkgc9fu3tu1cjmm721i2khqhz'}, 'href': 'https://api.spotify.com/v1/users/kkgc9fu3tu1cjmm721i2khqhz', 'id': 'kkgc9fu3tu1cjmm721i2khqhz', 'type': 'user', 'uri': 'spotify:user:kkgc9fu3tu1cjmm721i2khqhz'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6dgwEwnK0YtDfS9XhRwBTG'}, 'href': 'https://api.spotify.com/v1/artists/6dgwEwnK0YtDfS9XhRwBTG', 'id': '6dgwEwnK0YtDfS9XhRwBTG', 'name': 'Broken Bells', 'type': 'artist', 'uri': 'spotify:artist:6dgwEwnK0YtDfS9XhRwBTG'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 